In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D,AveragePooling2D, Flatten, concatenate, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import BatchNormalization


In [49]:
class Custom1_Conv2D(Layer):
    def __init__(self, n_filters, kernel_size, n_strides, padding='same'):
        super(Custom1_Conv2D, self).__init__(name='custom_conv2d')
        self.conv = Conv2D(filters=n_filters, kernel_size=kernel_size, strides=n_strides, padding=padding)
        self.bn = BatchNormalization()
        
    def call(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = Activation('relu')(x)
        return x

class InceptionModule(Layer):
    def __init__(self, reduce_3_X_3, org_3_X_3, reduce_5_X_5, org_5_X_5, org_pooling, org_1_X_1):
        super(InceptionModule, self).__init__(name='inception_module')
        
        
        self.conv1_reduce_3_X_3 = Custom1_Conv2D(reduce_3_X_3, 1, 1, 'same')
        self.conv2_reduce_5_X_5 = Custom1_Conv2D(reduce_5_X_5, 1, 1, 'same')
        self.maxpool_L1 = MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')
        
        self.conv1_3_X_3 = Custom1_Conv2D(org_3_X_3, 3, 1, 'same')
        self.conv2_5_X_5 = Custom1_Conv2D(org_5_X_5, 5, 1, 'same')
        self.conv_for_maxpool = Custom1_Conv2D(org_pooling, 1, 1, 'same')
        
        self.conv3 = Custom1_Conv2D(org_1_X_1, 1, 1, 'same')

    def call(self, input_):
        x1 = self.conv1_reduce_3_X_3(input_)
        x1_ = self.conv1_3_X_3(x1)

        x2 = self.conv2_reduce_5_X_5(input_)
        x2_ = self.conv2_5_X_5(x2)

        
        x3 = self.maxpool_L1(input_)
        x3_ = self.conv_for_maxpool(x3)

        
        x4_ = self.conv3(input_)
        
        
        return concatenate([x4_, x1_, x2_, x3_], axis=-1)

class Aux_Layers(Layer):
    def __init__(self):
        super(Aux_Layers, self).__init__(name='auxiliary_layers')
        
        self.averagepool = AveragePooling2D(pool_size=(5, 5), strides=3, padding='valid')
        self.conv = Custom1_Conv2D(128, 1, 1, 'same')
        self.flatten = Flatten()
        self.fc1 = Dense(1024, activation='relu')
        self.dropout = Dropout(0.7)
        self.fc2 = Dense(10, activation='softmax')  # Adjust to the number of classes

    def call(self, input_):
        x = self.averagepool(input_)
        x = self.conv(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class InceptionV1(Model):
    def __init__(self):
        super(InceptionV1, self).__init__(name='inceptionv1')
        
        #Custom Conv2D   -->   def __init__(self, n_filters, kernel_size, n_strides, padding='same'):
        #Inception Block -->   def __init__(self, reduce_3_X_3, org_3_X_3, reduce_5_X_5, org_5_X_5, org_pooling, org_1_X_1):
        #Instead of LRN layer, i have applied batch normalization in the Conv Layer itself 
        
        
        self.conv1 = Custom1_Conv2D(64, 7, 2,'same')
        self.maxpool1 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')
        self.conv2 = Custom1_Conv2D(64, 1, 1, 'valid')
        self.conv3 = Custom1_Conv2D(192, 1, 1)
        self.maxpool2 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')
        self.inception1 = InceptionModule(96,128,16,32,32,64)
        self.inception2 = InceptionModule(128,192,32,96,64,128)
        self.maxpool3 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')
        self.inception3 = InceptionModule(96,208,16,48,64,192)
        self.inception4 = InceptionModule(112,224,24,64,64,160)
        self.inception5 = InceptionModule(128,256,24,64,64,128)
        self.inception6 = InceptionModule(144,288,32,64,64,112)
        self.inception7 = InceptionModule(160,320,32,128,128,256)
        self.maxpool4 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')
        self.inception8 = InceptionModule(160,320,32,128,128,256)
        self.inception9 = InceptionModule(192,384,48,128,128,384)
        self.averagepool = AveragePooling2D(pool_size=(7, 7), strides=1, padding='valid')
        self.aux1 = Aux_Layers()
        self.aux2 = Aux_Layers()
        
        self.flatten = Flatten()
        self.dropout = Dropout(0.4)
        self.fc = Dense(10, activation='softmax')


    def call(self, input_):
        x = self.conv1(input_)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.maxpool3(x)
        x_1 = self.inception3(x)
        x = self.inception4(x_1)
        x = self.inception5(x)
        x_2 = self.inception6(x)
        x = self.inception7(x_2)
        x = self.maxpool4(x)
        x = self.inception8(x)
        x = self.inception9(x)
        x = self.averagepool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)

        aux1 = self.aux1(x_1)
        aux2 = self.aux2(x_2)

        return x, aux1, aux2

model = InceptionV1()

model.build(input_shape=(None, 224, 224, 3)) 
model.summary()


Model: "inceptionv1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_conv2d (Custom1_Conv  multiple                 9728      
 2D)                                                             
                                                                 
 max_pooling2d_280 (MaxPooli  multiple                 0         
 ng2D)                                                           
                                                                 
 custom_conv2d (Custom1_Conv  multiple                 4416      
 2D)                                                             
                                                                 
 custom_conv2d (Custom1_Conv  multiple                 13248     
 2D)                                                             
                                                                 
 max_pooling2d_281 (MaxPooli  multiple                 

In [1]:
# #Presently the following below code is taken from ChatGpt just for some random self testing purpose, but the rest of the code above is all mine 


# import tensorflow as tf
# import numpy as np
# from tensorflow.keras.losses import CategoricalCrossentropy
# from tensorflow.keras.optimizers import Adam


# model = InceptionV1()

# loss_function = CategoricalCrossentropy()
# optimizer = Adam()

# @tf.function
# def train_step(x, y):
#     with tf.GradientTape() as tape:
#         y_pred_main, y_pred_aux1, y_pred_aux2 = model(x)

#         loss_main = loss_function(y, y_pred_main)
#         loss_aux1 = loss_function(y, y_pred_aux1)
#         loss_aux2 = loss_function(y, y_pred_aux2)

#         total_loss = loss_main + 0.3 * loss_aux1 + 0.3 * loss_aux2 

#     gradients = tape.gradient(total_loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#     return total_loss

# num_samples = 10
# num_classes = 10
# input_shape = (224, 224, 3)

# x_train = np.random.random((num_samples,) + input_shape).astype(np.float32)
# y_train = tf.keras.utils.to_categorical(np.random.randint(num_classes, size=num_samples), num_classes)

# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# num_epochs = 2 

# for epoch in range(num_epochs):
#     for x, y in train_dataset:
#         loss = train_step(x[tf.newaxis, ...], y[tf.newaxis, ...]) 
#         print(f"Epoch {epoch}, Loss: {loss.numpy()}")
